In [1]:
from EEG2CodeKeras import (basearchi, basearchi_patchembedding,
                           basearchi_patchembeddingdilation, vanilliaEEG2Code,
                           trueVanilliaEEG2Code)
from tensorflow import keras
import tensorflow_addons as tfa
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import time
import sys
import shutil
from imblearn.under_sampling import RandomUnderSampler
import os
import mne
from collections import OrderedDict
from itertools import product
from keras.layers import Dense, Conv2D
from numba import cuda

from _utils import make_preds_accumul_aggresive, train_split
sys.path.insert(0,"C:\\Users\\s.velut\\Documents\\These\\Protheus_PHD\\Scripts")
from utils import prepare_data,get_BVEP_data,balance,get_y_pred


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If you were to en

In [2]:
print(tf.config.list_physical_devices())
# path = "D:/SET/SET/"
path = "~/Documents/These/Protheus_PHD/Class4"
n_cal_pretrain = 7
n_cal_finetuning = 5
learning_rate_pretrain = 1e-5
learning_rate_finetuning = 1e-3
batchsize = 64
# pre_train_list = np.array([1, 3, 5])
pre_train_list = np.array([1])
# , vanilliaEEG2Code2, basearchi_patchembeddingdilation
# models = {"BaseArchi": basearchi,
#           "PatchEmbedding": basearchi_patchembedding,
#           "EEG2Code": vanilliaEEG2Code,
#           "TrueEEG2Code": trueVanilliaEEG2Code}
n_class = 4
window_size = 0.25
fps = 60
pretraining_epochs = 20
finetuning_epochs = 20

min_len_possibilities = [5, 10, 20, 30, 40]
# consecutive_possibilities = [15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]
consecutive_possibilities = [10, 20, 30, 40, 50, 60]
LEARNING_RATE = 1e-3


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [3]:
def load_data(filepath):
    raw = mne.io.read_raw_eeglab(filepath, preload=True, verbose=False)
    # Average re-referencing
    # keep = ["O1", "O2", "Oz", "P7", "P3", "P4", "P8", "Pz"]
    # raw = raw.drop_channels([i for i in raw.ch_names if i not in keep])

    # to_drop = ['10', '21', 'ACC_X', 'ACC_Y', 'ACC_Z']
    to_drop = ["P9", "P10", "TP9", "TP10", "10", "21", 'ACC_X', 'ACC_Y', 'ACC_Z']
    raw = raw.drop_channels([ch for ch in raw.ch_names if ch in to_drop])

    raw = raw.filter(l_freq=50.1, h_freq=49.9, method="iir", verbose=False)
    mne.set_eeg_reference(raw, 'average', copy=False, verbose=False)
    # raw = raw.filter(l_freq=5, h_freq=45, method="fir", verbose=False)
    n_channels = len(raw.ch_names)
    # print("Channels :", n_channels)

    # Strip the annotations that were script to make them easier to process
    events, event_id = mne.events_from_annotations(
        raw, event_id='auto', verbose=False)
    to_remove = []
    for idx in range(len(raw.annotations.description)):
        if (('collects' in raw.annotations.description[idx]) or
            ('iti' in raw.annotations.description[idx]) or
                (raw.annotations.description[idx] == '[]')):
            to_remove.append(idx)
        else:
            code = raw.annotations.description[idx].split('_')[0]
            lab = raw.annotations.description[idx].split('_')[1]
            code = code.replace('\n', '')
            code = code.replace('[', '')
            code = code.replace(']', '')
            code = code.replace(' ', '')
            raw.annotations.description[idx] = code + '_' + lab

    to_remove = np.array(to_remove)
    if len(to_remove) > 0:
        raw.annotations.delete(to_remove)
    # Get the events
    events, event_id = mne.events_from_annotations(
        raw, event_id='auto', verbose=False)
    shift = 0.0
    # Epoch the data following event
    epochs = mne.Epochs(raw, events, event_id=event_id, tmin=shift,
                        tmax=2.2+shift, baseline=(None, None), preload=False, verbose=False)
    labels = epochs.events[..., -1]
    labels -= np.min(labels)
    data = epochs.get_data()

    codes = OrderedDict()
    for k, v in event_id.items():
        code = k.split('_')[0]
        code = code.replace('.', '').replace('2', '')
        idx = k.split('_')[1]
        codes[v-1] = np.array(list(map(int, code)))

    return data, labels, codes, int(epochs.info['sfreq']), n_channels


def test_split(data, labels, n_class, n_training_cal):
    data_train = data[:n_class*n_training_cal]
    labels_train = labels[:n_class*n_training_cal]
    data_test = data[n_class*n_training_cal:]
    labels_test = labels[n_class*n_training_cal:]

    return data_train, labels_train, data_test, labels_test


def to_window(data, labels, n_channels, w_size, codes, sfreq, fps):
    length = int((2.2-w_size)*sfreq)
    n_samples_windows = int(w_size*sfreq)
    X = np.empty(shape=((length)*data.shape[0], n_channels, n_samples_windows))
    Y = np.empty(shape=((length)*data.shape[0]), dtype=int)
    count = 0
    for trial_nb, trial in enumerate(data):
        lab = labels[trial_nb]
        c = codes[lab]
        code_pos = 0
        for idx in range(length):
            X[count] = trial[:, idx:idx+n_samples_windows]
            if idx/sfreq >= (code_pos+1)/fps:
                code_pos += 1
            Y[count] = int(c[code_pos])
            count += 1

    X = np.expand_dims(X, 1)
    X = X.astype(np.float32)
    Y = np.vstack((Y, np.abs(1-Y))).T
    return X, Y


def code2array(code):
    tmp = []
    for idx, c in enumerate(code[:-2]):
        if c == '5' or c == '.':
            continue
        elif c == '0':
            if code[idx+2] == '5':
                tmp.append(0.5)
            else:
                tmp.append(0)
        else:
            tmp.append(1)
    if code[-1] == '.':
        if code[-2] == '0':
            tmp.append(0)
        else:
            tmp.append(1)
    return np.array(tmp)


In [4]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import (
    InputLayer,
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    BatchNormalization,
    Normalization,
    Dropout,
    LeakyReLU
)


def base_norm(windows_size, n_channel_input, normlayer=None):

    model = Sequential()
    model.add(InputLayer(input_shape=(1, n_channel_input, windows_size)))
    if normlayer is not None:
        model.add(normlayer)
    model.add(
        Conv2D(
            16,
            kernel_size=(n_channel_input, 1),
            padding="valid",
            strides=(1, 1),
            data_format="channels_first",
            kernel_initializer="he_uniform",
            activation=None,
        )
    )
    model.add(BatchNormalization(axis=1, scale=True, center=False))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2),
              padding="same", data_format="channels_first"))
    model.add(Dropout(0.5))
    model.add(
        Conv2D(
            8,
            kernel_size=(1, 32),
            dilation_rate=(1, 2),
            data_format="channels_first",
            padding="same",
            kernel_initializer="he_uniform",
            activation=None,
        )
    )
    model.add(BatchNormalization(axis=1, scale=True, center=False))
    model.add(LeakyReLU(alpha=0.3))
    model.add(MaxPooling2D(pool_size=(1, 2), strides=(1, 2), padding="same"))
    model.add(Dropout(0.5))
    model.add(
        Conv2D(
            4,
            kernel_size=(5, 5),
            dilation_rate=(2, 2),
            data_format="channels_first",
            padding="same",
            kernel_initializer="he_uniform",
            activation=None,
        )
    )
    model.add(BatchNormalization(axis=1, scale=True, center=False))
    model.add(LeakyReLU(alpha=0.3))
    model.add(MaxPooling2D(pool_size=(2, 2),
              data_format="channels_first", padding="same"))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(int(256), activation=None))
    model.add(LeakyReLU(alpha=0.3))
    model.add(Dense(2, name="preds", activation="softmax"))
    return model


keras.backend.clear_session()

In [5]:
def make_preds_accumul_aggresive(y_pred, codes, min_len=30, sfreq=500, consecutive=30, window_size=0.25):
    length = int((2.2-window_size)*sfreq)
    y_pred = np.array(y_pred)
    rez_acc = []

    code_buffer = []
    labels_pred = []
    code_pos = 0
    y_tmp = [] 
    mean_long = []


    for trial in range(int(len(y_pred)/length)):   
        # Retrieve a trial
        tmp_code = y_pred[trial*length:(trial+1)*length]
        code_pos = 0

        # Do an average over the prdata, codes, labels, sfreq
        code_buffer = []
        for idx in range(len(tmp_code)):
            y_tmp.append(tmp_code[idx])
            if idx/sfreq >= (code_pos+1)/60:
                code_pred = np.mean(y_tmp) 
                code_pred = int(np.rint(code_pred))
                code_buffer.append(code_pred) 
                y_tmp = []
                code_pos += 1
        # Find the code that correlate the most
        pred_lab = -1
        out = 0
        for long in np.arange(min_len, len(code_buffer) -1 , step=1):
            dtw_values = []
            for key, values in codes.items():
                dtw_values.append(np.corrcoef(code_buffer[:long], values[:long])[0,1])
            dtw_values = np.array(dtw_values)
            max_dtw = list(codes.keys())[np.argmax(dtw_values)] 
            if (max_dtw == pred_lab):
                out += 1
            else:
                pred_lab = max_dtw
                out = 0
            if out == consecutive:
                mean_long.append((long)/60)
                break
        labels_pred.append(pred_lab)
    labels_pred = np.array(labels_pred)
    return labels_pred, code_buffer, mean_long

In [6]:
import math
import sklearn.metrics
from itertools import chain

def basearchi_StdFromXTrain(parti, X_train, Y_train, X_test, Y_test, n_samples_windows, n_channels, labels_test, codes, sfreq, ctype):
    X_std = X_train.std(axis=0)
    X_train_final = X_train /(X_std + 1e-8)
    X_test_final = X_test / (X_std + 1e-8)
    clf = basearchi(windows_size=n_samples_windows, n_channel_input=n_channels)
    
    return predict(parti, "BaseArchi_XTestNormByXTrain", clf, X_train_final, Y_train, X_test_final, Y_test, labels_test, codes, sfreq, ctype)

def basearchi_StdFromXTest(parti, X_train, Y_train, X_test, Y_test, n_samples_windows, n_channels, labels_test, codes, sfreq, ctype):
    X_std = X_train.std(axis=0)
    X_train_final = X_train /(X_std + 1e-8)
    X_std = X_test.std(axis=0)
    X_test_final = X_test / (X_std + 1e-8)
    clf = basearchi(windows_size=n_samples_windows, n_channel_input=n_channels)

    return predict(parti, "BaseArchi_XTestNormByXTest", clf, X_train_final, Y_train, X_test_final, Y_test, labels_test, codes, sfreq, ctype)

def basearchi_GlobalNorm(parti, X_train, Y_train, X_test, Y_test, n_samples_windows, n_channels, labels_test, codes, sfreq, ctype):
    normlayer = Normalization(axis=None)
    normlayer.adapt(X_train)
    clf = base_norm(windows_size=n_samples_windows,
                    n_channel_input=n_channels, normlayer=normlayer)

    return predict(parti, "Global Norm", clf, X_train, Y_train, X_test, Y_test, labels_test, codes, sfreq, ctype)

def basearchi_SampleNorm(parti, X_train, Y_train, X_test, Y_test, n_samples_windows, n_channels, labels_test, codes, sfreq, ctype):
    normlayer = Normalization(axis=0)
    normlayer.adapt(X_train)
    clf = base_norm(windows_size=n_samples_windows,
                    n_channel_input=n_channels, normlayer=normlayer)

    return predict(parti, "Sample Norm", clf, X_train, Y_train, X_test, Y_test, labels_test, codes, sfreq, ctype)

def basearchi_Norm_ElectrodeNorm(parti, X_train, Y_train, X_test, Y_test, domains_train, nb_seconds,
                             n_samples_windows, n_channels, labels_test, codes, sfreq, ctype):
    
    for d in np.unique(domains_train):
        normlayer = Normalization(axis=-2)
        normlayer.adapt(X_train[domains_train==d])
        X_train[domains_train==d] = normlayer(X_train[domains_train==d])
    
    normlayer = Normalization(axis=-2)
    normlayer.adapt(X_test[:nb_seconds*60])
    X_test = normlayer(X_test)

    clf = base_norm(windows_size=n_samples_windows,
                    n_channel_input=n_channels, normlayer=None)

    return predict(parti, "Normed Electrode Norm", clf, X_train, Y_train, X_test, Y_test, labels_test, codes, sfreq, ctype)

def basearchi_ElectrodeNorm(parti, X_train, Y_train, X_test, Y_test, n_samples_windows, n_channels, labels_test, codes, sfreq, ctype):
    normlayer = Normalization(axis=-2)
    normlayer.adapt(X_train)
    clf = base_norm(windows_size=n_samples_windows,
                    n_channel_input=n_channels, normlayer=normlayer)

    return predict(parti, "Electrode Norm", clf, X_train, Y_train, X_test, Y_test, labels_test, codes, sfreq, ctype)

def basearchi_TemporalNorm(parti, X_train, Y_train, X_test, Y_test, n_samples_windows, n_channels, labels_test, codes, sfreq, ctype):
    normlayer = Normalization(axis=-1)
    normlayer.adapt(X_train)
    clf = base_norm(windows_size=n_samples_windows,
                    n_channel_input=n_channels, normlayer=normlayer)

    return predict(parti, "Temporal Norm",clf, X_train, Y_train, X_test, Y_test, labels_test, codes, sfreq, ctype)


def predict(parti, model_str, model, X_train, Y_train, X_test, Y_test, labels_test, codes, sfreq, ctype):
    X_train_tr, X_val, Y_train_tr, Y_val = train_test_split(
        X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE, amsgrad=True)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    training_time = time.time()
    print("    * Training model")
    print(np.array(X_train_tr.shape))
    print(Y_train_tr.shape)
    history = model.fit(
        np.array(X_train_tr),
        Y_train_tr,
        batch_size=128,
        epochs=35,
        validation_data=(np.array(X_val), Y_val),
        verbose=0,
        shuffle=True
    )

    training_time = time.time()-training_time

    print("    * Prediciting model")
    y_pred = model.predict(X_test, batch_size=24, verbose=0)[:, 0]
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([0 if y[0] == 0 else 1 for y in Y_test])
    tn, fp, fn, tp = sklearn.metrics.confusion_matrix(
        y_test_norm, y_pred_norm).ravel()
    test_acc = (tp+tn)/len(y_test_norm)
    test_sensitivity = tp/(tp+fn)
    test_precision = tp/(tp+fp)
    test_fowlkes_mallows = math.sqrt(test_sensitivity*test_precision)

    acc_best = 0
    acc_best_c = 0
    acc_best_m = 0
    itr_best = 0
    itr_best_c = 0
    itr_best_m = 0
    itr_best_acc = 0
    file_name = f"./results/test_norm/user{parti}_{model_str}_{ctype}.csv"
    class_results = pd.DataFrame(columns=list(chain(*map(lambda i: ["Acc"+i, "Time"+i, "ITR"+i], map(str, min_len_possibilities)))))
    
    print("    * Computing windows scores")
    for c in consecutive_possibilities:
        temp_res = []
        for m in min_len_possibilities:
            labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
                y_pred_norm, codes, min_len=m, sfreq=sfreq, consecutive=c, window_size=window_size
            )
            accuracy = round(accuracy_score(
                labels_test[labels_pred_accumul != -1], labels_pred_accumul[labels_pred_accumul != -1]), 2)
            if accuracy == 0:
                itr = 0
            elif accuracy == 1.0:
                itr = (math.log2(n_class)+accuracy*math.log2(accuracy))*60/np.mean(mean_long_accumul)
            else: 
                itr = round((math.log2(n_class)+accuracy*math.log2(accuracy)+(1-accuracy)*math.log2((1-accuracy)/(n_class-1)))*60/np.mean(mean_long_accumul), 0)
            if accuracy > acc_best:
                acc_best_m = m
                acc_best_c = c
                acc_best = accuracy
                acc_best_itr = itr

            if itr > itr_best:
                itr_best_m = m
                itr_best_c = c
                itr_best_acc = accuracy
                itr_best = itr
            temp_res.extend([accuracy, np.mean(mean_long_accumul), itr])
        class_results.loc[len(class_results)] = temp_res
        class_results = class_results.rename(index={class_results.index[-1]: str()})

    class_results.to_csv(file_name)
    del class_results, model, temp_res
    keras.backend.clear_session()
    keras.backend.clear_session()
    keras.backend.clear_session()
    keras.backend.clear_session()

    return [parti, ctype, model_str, history.history["accuracy"][-1], history.history["val_accuracy"][-1],
            training_time, tp, fp, tn, fn, test_acc, test_sensitivity, test_precision,
            test_fowlkes_mallows, itr_best_m, itr_best_c, itr_best_acc, itr_best, acc_best_m,
            acc_best_c, acc_best, acc_best_itr, file_name]


### Test on normalise train test

In [8]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

PARTICIPANT = "Participant"
CODE_TYPE = "CodeType"
MODEL_TYPE = "ModelType"
CNN_TRAINING_ACC = "CnnTrainingAccuracy"
CNN_TRAINING_VALACC = "CnnTrainingVALAccuracy"
CNN_TRAINING_TIME = "CnnTrainingTime"
CNN_TRUE_POSITIVES = "CnnTestTruePositive"
CNN_FALSE_POSITIVES = "CnnTestFalsePositive"
CNN_TRUE_NEGATIVES = "CnnTestTrueNegatives"
CNN_FALSE_NEGATIVES = "CnnTestFalseNegatives"
CNN_ACCURACY = "CnnTestAccuracy"
CNN_SENSITIVITY = "CnnTestSensitivity"
CNN_PRECISION = "CnnTestPrecision"
CNN_FOWLKES_MALLOWS = "CnnTestFowlkesMallows"
CLASS_RESULTS_FILE = "ClassWindowResultFile"
CLASS_RESULTS_BEST_MIN_LEN_BY_ITR = "ClassWindowResultBestMinLenByITR"
CLASS_RESULTS_BEST_CONSECUTIVE_BY_ITR = "ClassWindowResultBestConsecutiveByITR"
CLASS_RESULTS_BEST_ACC_BY_ITR = "ClassWindowResultBestAccByITR"
CLASS_RESULTS_BEST_ITR_BY_ITR = "ClassWindowResultBestITRByITR"
CLASS_RESULTS_BEST_MIN_LEN_BY_ACC = "ClassWindowResultBestMinLenByAccuracy"
CLASS_RESULTS_BEST_CONSECUTIVE_BY_ACC = "ClassWindowResultBestConsecutiveByAccuracy"
CLASS_RESULTS_BEST_ACC_BY_ACC = "ClassWindowResultBestAccByAccuracy"
CLASS_RESULTS_BEST_ITR_BY_ACC = "ClassWindowResultBestITRByAccuracy"

results = pd.DataFrame(columns=[PARTICIPANT, CODE_TYPE, MODEL_TYPE, CNN_TRAINING_ACC, CNN_TRAINING_VALACC, CNN_TRAINING_TIME,
                                CNN_TRUE_POSITIVES, CNN_FALSE_POSITIVES, CNN_TRUE_NEGATIVES,
                                CNN_FALSE_NEGATIVES, CNN_ACCURACY, CNN_SENSITIVITY, CNN_PRECISION,
                                CNN_FOWLKES_MALLOWS, CLASS_RESULTS_BEST_MIN_LEN_BY_ITR,
                                CLASS_RESULTS_BEST_CONSECUTIVE_BY_ITR, CLASS_RESULTS_BEST_ACC_BY_ITR,
                                CLASS_RESULTS_BEST_ITR_BY_ITR, CLASS_RESULTS_BEST_MIN_LEN_BY_ACC,
                                CLASS_RESULTS_BEST_CONSECUTIVE_BY_ACC, CLASS_RESULTS_BEST_ACC_BY_ACC,
                                CLASS_RESULTS_BEST_ITR_BY_ACC, CLASS_RESULTS_FILE])

sfreq = 500

#for ctype in ["burst100", "burst40", "mseq100", "mseq40"]:
for ctype in ["burst100"]:
    print("==================================================")
    print("================= TYPE", ctype, "=================")
    print("==================================================")
    for sub in range(1,14):
        print("================= USER", sub, "=================")
        data, labels, codes, sfreq, n_channels = load_data(
            os.path.join(path, f"P{sub}", f"P{sub}_{ctype}.set"))
        data_train, labels_train, data_test, labels_test = test_split(
            data, labels, n_class, n_cal_pretrain)
        X_train, Y_train = to_window(
            data_train, labels_train, n_channels, window_size, codes, sfreq, fps)
        X_test, Y_test = to_window(data_test, labels_test,
                                n_channels, window_size, codes, sfreq, fps)
        rus = RandomUnderSampler()
        counter = np.array(range(0, len(Y_train))).reshape(-1, 1)
        index, _ = rus.fit_resample(counter, Y_train[:, 0])
        X_train_ru = np.squeeze(X_train[index, :, :, :], axis=1)
        Y_train_ru = np.squeeze(Y_train[index])

        n_samples_windows = X_train.shape[-1]
        n_channels = X_train.shape[-2]
        print(" * BaseArchi_XTestNormByXTrain")
        results.loc[len(results)] = basearchi_StdFromXTrain(sub, X_train_ru, Y_train_ru, X_test, Y_test, n_samples_windows, n_channels, labels_test, codes, sfreq, ctype)
        results.to_csv("./results/final_results.csv")
        print(" * BaseArchi_XTestNormByXTest")
        results.loc[len(results)] = basearchi_StdFromXTest(sub, X_train_ru, Y_train_ru, X_test, Y_test, n_samples_windows, n_channels, labels_test, codes, sfreq, ctype)
        results.to_csv("./results/final_results.csv")
        print(" * Global Norm")
        results.loc[len(results)] = basearchi_GlobalNorm(sub, X_train_ru, Y_train_ru, X_test, Y_test, n_samples_windows, n_channels, labels_test, codes, sfreq, ctype)
        results.to_csv("./results/final_results.csv")
        print(" * Electrode Norm")
        results.loc[len(results)] = basearchi_ElectrodeNorm(sub, X_train_ru, Y_train_ru, X_test, Y_test, n_samples_windows, n_channels, labels_test, codes, sfreq, ctype)
        results.to_csv("./results/final_results.csv")
        print(" * Normed Electrode Norm")
        results.loc[len(results)] = basearchi_Norm_ElectrodeNorm(sub, X_train_ru, Y_train_ru, X_test, Y_test, ["train"], 5,
                                                                  n_samples_windows, n_channels, labels_test, codes, sfreq, ctype)
        results.to_csv("./results/final_results.csv")
        print(" * Temporal Norm")
        results.loc[len(results)] = basearchi_TemporalNorm(sub, X_train_ru, Y_train_ru,X_test, Y_test, n_samples_windows, n_channels, labels_test, codes, sfreq, ctype)
        results.to_csv("./results/final_results.csv")


================= TYPE burst100 =================
================= USER 1 =================


Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
 * BaseArchi_XTestNormByXTrain

    * Training model
[8416    1   30  125]
(8416, 2)




KeyboardInterrupt: 

## Normalised by domain LOO

In [7]:
subjects = [1,2,3,4,5,6,7,8,9,10,11,12]
to_drop = ["P9", "P10", "TP9", "TP10", "10", "21", 'ACC_X', 'ACC_Y', 'ACC_Z']
on_frame = False
raw_data,nlabels,codes,labels_codes = get_BVEP_data(subjects,on_frame,to_drop)
data, labels, domains = prepare_data(subjects,raw_data,nlabels,on_frame,False,codes)
# data = []
# labels = []
# domains = []

# for s in subjects:
#     print(s)
#     temp_data,temp_labels,temp_domains = prepare_data([s],raw_data,nlabels,on_frame,False,codes)
#     data.append(temp_data[0])
#     labels.append(temp_labels[0])
#     domains.append(temp_domains[0])
# domains = np.array(domains)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 50 - 50 Hz

IIR filter parameters
---------------------
Butterworth bandstop zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 49.90, 50.10 Hz: -6.02, -6.02 dB

Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 50 - 50 Hz

IIR filter parameters
---------------------
Butterworth bandstop zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 49.90, 50.10 Hz: -6.02, -6.02 dB

Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 50 - 50 Hz

IIR filter parameters
---------------------
Butterworth bandstop zero-phase (two-pas

### On sample

In [8]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

PARTICIPANT = "Participant"
CODE_TYPE = "CodeType"
MODEL_TYPE = "ModelType"
CNN_TRAINING_ACC = "CnnTrainingAccuracy"
CNN_TRAINING_VALACC = "CnnTrainingVALAccuracy"
CNN_TRAINING_TIME = "CnnTrainingTime"
CNN_TRUE_POSITIVES = "CnnTestTruePositive"
CNN_FALSE_POSITIVES = "CnnTestFalsePositive"
CNN_TRUE_NEGATIVES = "CnnTestTrueNegatives"
CNN_FALSE_NEGATIVES = "CnnTestFalseNegatives"
CNN_ACCURACY = "CnnTestAccuracy"
CNN_SENSITIVITY = "CnnTestSensitivity"
CNN_PRECISION = "CnnTestPrecision"
CNN_FOWLKES_MALLOWS = "CnnTestFowlkesMallows"
CLASS_RESULTS_FILE = "ClassWindowResultFile"
CLASS_RESULTS_BEST_MIN_LEN_BY_ITR = "ClassWindowResultBestMinLenByITR"
CLASS_RESULTS_BEST_CONSECUTIVE_BY_ITR = "ClassWindowResultBestConsecutiveByITR"
CLASS_RESULTS_BEST_ACC_BY_ITR = "ClassWindowResultBestAccByITR"
CLASS_RESULTS_BEST_ITR_BY_ITR = "ClassWindowResultBestITRByITR"
CLASS_RESULTS_BEST_MIN_LEN_BY_ACC = "ClassWindowResultBestMinLenByAccuracy"
CLASS_RESULTS_BEST_CONSECUTIVE_BY_ACC = "ClassWindowResultBestConsecutiveByAccuracy"
CLASS_RESULTS_BEST_ACC_BY_ACC = "ClassWindowResultBestAccByAccuracy"
CLASS_RESULTS_BEST_ITR_BY_ACC = "ClassWindowResultBestITRByAccuracy"

results = pd.DataFrame(columns=[PARTICIPANT, CODE_TYPE, MODEL_TYPE, CNN_TRAINING_ACC, CNN_TRAINING_VALACC, CNN_TRAINING_TIME,
                                CNN_TRUE_POSITIVES, CNN_FALSE_POSITIVES, CNN_TRUE_NEGATIVES,
                                CNN_FALSE_NEGATIVES, CNN_ACCURACY, CNN_SENSITIVITY, CNN_PRECISION,
                                CNN_FOWLKES_MALLOWS, CLASS_RESULTS_BEST_MIN_LEN_BY_ITR,
                                CLASS_RESULTS_BEST_CONSECUTIVE_BY_ITR, CLASS_RESULTS_BEST_ACC_BY_ITR,
                                CLASS_RESULTS_BEST_ITR_BY_ITR, CLASS_RESULTS_BEST_MIN_LEN_BY_ACC,
                                CLASS_RESULTS_BEST_CONSECUTIVE_BY_ACC, CLASS_RESULTS_BEST_ACC_BY_ACC,
                                CLASS_RESULTS_BEST_ITR_BY_ACC, CLASS_RESULTS_FILE])
sfreq=500
#for ctype in ["burst100", "burst40", "mseq100", "mseq40"]:
for ctype in ["burst100"]:
    print("==================================================")
    print("================= TYPE", ctype, "=================")
    print("==================================================")
    for sub in range(0,12):
        print("================= USER", sub, "=================")
        ind2take = [j for j in range(12) if j!=sub]
        # X_train = data[ind2take[0]]
        # Y_train = labels[ind2take[0]]
        # for k in ind2take[1:]:
        #     X_train += data[k]
        #     print(len(X_train))
        #     Y_train += labels[k]
        # print(len(Y_train))

        # X_train = np.expand_dims(X_train,1)
        X_train = np.expand_dims(np.concatenate(data[ind2take]).reshape(-1,data.shape[-2],data.shape[-1]),1)
        Y_train = np.concatenate(labels[ind2take]).reshape(-1)
        Y_train = np.vstack((Y_train,np.abs(1-Y_train))).T
        domains_train = np.concatenate(domains[ind2take]).reshape(-1)
        X_test = np.expand_dims(data[sub],1)
        Y_test = np.vstack((labels[sub],np.abs(1-labels[sub]))).T
        domains_test = domains[sub]
        
        X_train_ru, Y_train_ru,domain_train_ru = balance(X_train,Y_train,domains_train)
        
        n_samples_windows = X_train.shape[-1]
        n_channels = X_train.shape[-2]
        print(" * Global Norm")
        results.loc[len(results)] = basearchi_GlobalNorm(sub, X_train_ru, Y_train_ru, X_test, Y_test, n_samples_windows, n_channels, labels_codes[sub], codes, sfreq, ctype)
        results.to_csv("./results/moabb_final_results_sample.csv")
        print(" * Electrode Norm")
        results.loc[len(results)] = basearchi_ElectrodeNorm(sub, X_train_ru, Y_train_ru, X_test, Y_test, n_samples_windows, n_channels, labels_codes[sub], codes, sfreq, ctype)
        results.to_csv("./results/moabb_final_results_sample.csv")
        print(" * Normed Electrode Norm")
        results.loc[len(results)] = basearchi_Norm_ElectrodeNorm(sub, X_train_ru, Y_train_ru, X_test, Y_test, domain_train_ru, 25,
                                                                  n_samples_windows, n_channels, labels_codes[sub], codes, sfreq, ctype)
        results.to_csv("./results/moabb_final_results_sample.csv")
        print(" * Temporal Norm")
        results.loc[len(results)] = basearchi_StdFromXTest(sub, X_train_ru, Y_train_ru,X_test, Y_test, n_samples_windows, n_channels, labels_codes[sub], codes, sfreq, ctype)
        results.to_csv("./results/moabb_final_results_sample.csv")
        # print(results)

        del X_train
        del X_test
        del Y_train
        del Y_test
        del domains_train
        del domains_test
        


================= TYPE burst100 =================
================= USER 0 =================
 * Global Norm

    * Training model
[198396      1     30    125]
(198396, 2)


    * Prediciting model
    * Computing windows scores
 * Electrode Norm
    * Training model
[198396      1     30    125]
(198396, 2)
    * Prediciting model
    * Computing windows scores
 * Normed Electrode Norm
    * Training model
[198396      1     30    125]
(198396, 2)
    * Prediciting model
    * Computing windows scores
 * Temporal Norm
    * Training model
[198396      1     30    125]
(198396, 2)
    * Prediciting model
    * Computing windows scores
================= USER 1 =================
 * Global Norm
    * Training model
[198396      1     30    125]
(198396, 2)
    * Prediciting model
    * Computing windows scores
 * Electrode Norm
    * Training model
[198396      1     30    125]
(198396, 2)
    * Prediciting model
    * Computing windows scores
 * Normed Electrode Norm
    * Training model

## On frame

In [10]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

PARTICIPANT = "Participant"
CODE_TYPE = "CodeType"
MODEL_TYPE = "ModelType"
CNN_TRAINING_ACC = "CnnTrainingAccuracy"
CNN_TRAINING_VALACC = "CnnTrainingVALAccuracy"
CNN_TRAINING_TIME = "CnnTrainingTime"
CNN_TRUE_POSITIVES = "CnnTestTruePositive"
CNN_FALSE_POSITIVES = "CnnTestFalsePositive"
CNN_TRUE_NEGATIVES = "CnnTestTrueNegatives"
CNN_FALSE_NEGATIVES = "CnnTestFalseNegatives"
CNN_ACCURACY = "CnnTestAccuracy"
CNN_SENSITIVITY = "CnnTestSensitivity"
CNN_PRECISION = "CnnTestPrecision"
CNN_FOWLKES_MALLOWS = "CnnTestFowlkesMallows"
CLASS_RESULTS_FILE = "ClassWindowResultFile"
CLASS_RESULTS_BEST_MIN_LEN_BY_ITR = "ClassWindowResultBestMinLenByITR"
CLASS_RESULTS_BEST_CONSECUTIVE_BY_ITR = "ClassWindowResultBestConsecutiveByITR"
CLASS_RESULTS_BEST_ACC_BY_ITR = "ClassWindowResultBestAccByITR"
CLASS_RESULTS_BEST_ITR_BY_ITR = "ClassWindowResultBestITRByITR"
CLASS_RESULTS_BEST_MIN_LEN_BY_ACC = "ClassWindowResultBestMinLenByAccuracy"
CLASS_RESULTS_BEST_CONSECUTIVE_BY_ACC = "ClassWindowResultBestConsecutiveByAccuracy"
CLASS_RESULTS_BEST_ACC_BY_ACC = "ClassWindowResultBestAccByAccuracy"
CLASS_RESULTS_BEST_ITR_BY_ACC = "ClassWindowResultBestITRByAccuracy"

results = pd.DataFrame(columns=[PARTICIPANT, CODE_TYPE, MODEL_TYPE, CNN_TRAINING_ACC, CNN_TRAINING_VALACC, CNN_TRAINING_TIME,
                                CNN_TRUE_POSITIVES, CNN_FALSE_POSITIVES, CNN_TRUE_NEGATIVES,
                                CNN_FALSE_NEGATIVES, CNN_ACCURACY, CNN_SENSITIVITY, CNN_PRECISION,
                                CNN_FOWLKES_MALLOWS, CLASS_RESULTS_BEST_MIN_LEN_BY_ITR,
                                CLASS_RESULTS_BEST_CONSECUTIVE_BY_ITR, CLASS_RESULTS_BEST_ACC_BY_ITR,
                                CLASS_RESULTS_BEST_ITR_BY_ITR, CLASS_RESULTS_BEST_MIN_LEN_BY_ACC,
                                CLASS_RESULTS_BEST_CONSECUTIVE_BY_ACC, CLASS_RESULTS_BEST_ACC_BY_ACC,
                                CLASS_RESULTS_BEST_ITR_BY_ACC, CLASS_RESULTS_FILE])
sfreq=500
#for ctype in ["burst100", "burst40", "mseq100", "mseq40"]:
for ctype in ["burst100"]:
    print("==================================================")
    print("================= TYPE", ctype, "=================")
    print("==================================================")
    for sub in range(0,12):
        print("================= USER", sub, "=================")
        ind2take = [j for j in range(12) if j!=sub]
        X_train = np.expand_dims(np.concatenate(data[ind2take]).reshape(-1,data.shape[-2],data.shape[-1]),1)
        Y_train = np.concatenate(labels[ind2take]).reshape(-1)
        Y_train = np.vstack((Y_train,np.abs(1-Y_train))).T
        domains_train = np.concatenate(domains[ind2take]).reshape(-1)
        X_test = np.expand_dims(data[sub],1)
        Y_test = np.vstack((labels[sub],np.abs(1-labels[sub]))).T
        domains_test = domains[sub]
        
        X_train_ru, Y_train_ru,domain_train_ru = balance(X_train,Y_train,domains_train)
        
        n_samples_windows = X_train.shape[-1]
        n_channels = X_train.shape[-2]
        print(" * Global Norm")
        results.loc[len(results)] = basearchi_GlobalNorm(sub, X_train_ru, Y_train_ru, X_test, Y_test, n_samples_windows, n_channels, labels_codes[sub], codes, 60, ctype)
        results.to_csv("./results/moabb_final_results_frame.csv")
        print(" * Electrode Norm")
        results.loc[len(results)] = basearchi_ElectrodeNorm(sub, X_train_ru, Y_train_ru, X_test, Y_test, n_samples_windows, n_channels, labels_codes[sub], codes, 60, ctype)
        results.to_csv("./results/moabb_final_results_frame.csv")
        print(" * Normed Electrode Norm")
        results.loc[len(results)] = basearchi_Norm_ElectrodeNorm(sub, X_train_ru, Y_train_ru, X_test, Y_test, domain_train_ru, 5,
                                                                  n_samples_windows, n_channels, labels_codes[sub], codes, 60, ctype)
        results.to_csv("./results/moabb_final_results_frame.csv")
        print(" * Temporal Norm")
        results.loc[len(results)] = basearchi_TemporalNorm(sub, X_train_ru, Y_train_ru,X_test, Y_test, n_samples_windows, n_channels, labels_codes[sub], codes, 60, ctype)
        results.to_csv("./results/moabb_final_results_frame.csv")
        # print(results)

        


================= TYPE burst100 =================
================= USER 0 =================


TypeError: list indices must be integers or slices, not list

In [17]:
df = pd.read_csv("./results/moabb_final_results2.csv")
globres = df.loc[df["ModelType"]=="Global Norm"][["Participant","CnnTrainingAccuracy","CnnTrainingVALAccuracy","CnnTestAccuracy","ClassWindowResultBestAccByAccuracy"]]
electres = df.loc[df["ModelType"]=="Electrode Norm"][["Participant","CnnTrainingAccuracy","CnnTrainingVALAccuracy","CnnTestAccuracy","ClassWindowResultBestAccByAccuracy"]]
normelectres = df.loc[df["ModelType"]=="Normed Electrode Norm"][["Participant","CnnTrainingAccuracy","CnnTrainingVALAccuracy","CnnTestAccuracy","ClassWindowResultBestAccByAccuracy"]]
tempres = df.loc[df["ModelType"]=="Temporal Norm"][["Participant","CnnTrainingAccuracy","CnnTrainingVALAccuracy","CnnTestAccuracy","ClassWindowResultBestAccByAccuracy"]]

In [20]:
print(np.mean(globres["ClassWindowResultBestAccByAccuracy"]))
print(np.mean(electres["ClassWindowResultBestAccByAccuracy"]))
print(np.mean(normelectres["ClassWindowResultBestAccByAccuracy"]))
print(np.mean(tempres["ClassWindowResultBestAccByAccuracy"]))


0.8075000000000001
0.8041666666666667
0.8474999999999998
0.25


In [25]:
df = pd.read_csv("./results/moabb_final_results.csv",sep=";")
globres = df.loc[df["ModelType"]=="Global Norm"][["Participant","CnnTrainingAccuracy","CnnTrainingVALAccuracy","CnnTestAccuracy","ClassWindowResultBestAccByAccuracy"]]
electres = df.loc[df["ModelType"]=="Electrode Norm"][["Participant","CnnTrainingAccuracy","CnnTrainingVALAccuracy","CnnTestAccuracy","ClassWindowResultBestAccByAccuracy"]]
normelectres = df.loc[df["ModelType"]=="Normed Electrode Norm"][["Participant","CnnTrainingAccuracy","CnnTrainingVALAccuracy","CnnTestAccuracy","ClassWindowResultBestAccByAccuracy"]]
tempres = df.loc[df["ModelType"]=="Temporal Norm"][["Participant","CnnTrainingAccuracy","CnnTrainingVALAccuracy","CnnTestAccuracy","ClassWindowResultBestAccByAccuracy"]]

In [26]:
print(np.mean(globres["ClassWindowResultBestAccByAccuracy"]))
print(np.mean(electres["ClassWindowResultBestAccByAccuracy"]))
print(np.mean(normelectres["ClassWindowResultBestAccByAccuracy"]))
print(np.mean(tempres["ClassWindowResultBestAccByAccuracy"]))


0.8391666666666667
0.8283333333333333
0.8358333333333334
0.8391666666666667
